### Imports

In [6]:
import importlib
import traceback
import os
import re
from tqdm import tqdm

import findatree.io as io
import findatree.photometric as photometric
import findatree.transformations as transformations

_ = importlib.reload(io)
_ = importlib.reload(photometric)
_ = importlib.reload(transformations)

### Definitions

In [10]:
# Define full paths to directories containing dtm, dsm, ortho and shape-files 
dir_names=[]
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\DSMs'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\DTMs'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\Orthos'])
dir_names.extend([r'C:\Data\lwf\WZE-UAV-2020\Kronen'])

# Define directories where return hdf5s are stored
dir_name_hdf5 = r"C:\Data\lwf\processed\2020\hdf5"

# Optionally process only specific tnrs
tnrs_specific = [
    # 10547,
]

### Processing

#### Prepare main loop

In [11]:
# Get all available tnr numbers in directory
tnrs = io.find_all_tnrs_in_dir(dir_name_hdf5, tnr_pattern_lead='tnr', tnr_pattern_trail='.hdf5')

# Define full paths to all pre-processed tnr hdf5s
paths = [os.path.join(dir_name_hdf5, f"tnr{tnr}.hdf5") for tnr in tnrs]


# Include only specified tnrs in main loop
if len(tnrs_specific) > 0:
    tnrs_keep_idx = tnrs.index(tnrs_specific)
    
    try:
        paths_keep = [paths[idx] for idx in tnrs_keep_idx]
    except:
        paths_keep = [paths[tnrs_keep_idx]]

    print('Following files will be included in main loop:')
    for p in paths_keep: print(' '*3, p)

else:
    tnrs_keep = tnrs.copy()
    paths_keep = paths.copy()

#### Run main loop

In [12]:
exception_traceback = []

for path in tqdm(paths_keep):
    
    try:
        
        ################# [1] Load channels and crowns group from hdf5
        
        data, params_data = io.load_hdf5(path, groups = ['channels'])
        channels, params_channels = data['channels'].copy(), params_data['channels'].copy()
        del data, params_data


        ################# [2] Load and save shapefile to correct tnr and save in hdf5 under group 'crowns_human'
        
        crowns_human, params_crowns_human = io.load_shapefile(dir_names, params_channels, verbose=False)
        photometric.crowns_add_features(
            channels,
            params_channels,
            crowns_human,
            params_crowns_human,
        )
        io.crowns_to_hdf5(crowns_human, params_crowns_human, dir_name = dir_name_hdf5)

    except:
        exception_traceback.append({
            'path': path,
            'traceback': traceback.format_exc(),
            })


# Save exceptions in .yaml
io.list_of_dicts_to_yaml(
    path = os.path.join(dir_name_hdf5, f"{transformations.current_datetime()}_batch_update_crowns_human.yaml"),
    list_of_dicts = exception_traceback,
    )

print()
print('Done!')
print(f"{len(exception_traceback)} exceptions occured")

100%|██████████| 128/128 [08:02<00:00,  3.77s/it]


Done!
2 exceptions occured


In [9]:
for e in exception_traceback:
    print(e['path'])
    print('-->', e['traceback'])